# Value gradient error for linear policies in LQG

Experiment description on [Overleaf](https://www.overleaf.com/read/cmbgmxxpxqzr).

**Versioning:** [CalVer](https://calver.org) `MM.DD.MICRO`

In [1]:
from __future__ import annotations

import logging
import os
import os.path as osp

import lqsvg
import lqsvg.envs.lqr.utils as lqg_util
import lqsvg.experiment.utils as utils
import lqsvg.torch.named as nt
import pytorch_lightning as pl
import ray
from lqsvg.experiment.data import build_datamodule
from lqsvg.experiment.models import LightningModel
from lqsvg.experiment.worker import make_worker
from ray import tune
from raylab.policy.model_based.lightning import LightningTrainerSpec
from torch import Tensor

import wandb

In [2]:
CWD = os.getcwd()

In [3]:
class InputStatistics(pl.callbacks.Callback):
    def on_train_batch_end(
        self,
        trainer: pl.Trainer,
        pl_module: pl.LightningModule,
        outputs: Tensor,
        batch: tuple[Tensor, Tensor, Tensor],
        batch_idx: int,
        dataloader_idx: int,
    ):
        del trainer, outputs, batch_idx, dataloader_idx
        obs, act, new_obs = batch
        pl_module.log("train/obs-mean", obs.mean())
        pl_module.log("train/obs-std", obs.std())
        pl_module.log("train/act-mean", act.mean())
        pl_module.log("train/act-std", act.std())
        pl_module.log("train/new_obs-mean", new_obs.mean())
        pl_module.log("train/new_obs-std", new_obs.std())

In [4]:
class Experiment(tune.Trainable):
    def setup(self, config: dict):
        self.run = wandb.init(
            dir=osp.join(CWD, ""),
            name="SVG Prediction",
            config=config,
            project="LQG-SVG",
            entity="angelovtt",
            tags=[utils.calver()],
            reinit=True,
            mode="online",
            save_code=True,
        )

        self.make_worker()
        self.make_model()
        self.make_datamodule()
        self.make_lightning_trainer()
        self.make_artifact()
        utils.suppress_lightning_info_logging()

    @property
    def hparams(self):
        return self.run.config

    def make_worker(self):
        with nt.suppress_named_tensor_warning():
            self.worker = make_worker(
                env_config=self.hparams.env_config,
                policy_config=self.hparams.policy,
                log_level=logging.WARNING,
            )

    def make_model(self):
        self.model = LightningModel(self.worker.get_policy(), self.worker.env)
        self.model.hparams.learning_rate = self.hparams.learning_rate
        self.model.hparams.mc_samples = self.hparams.mc_samples

    def make_datamodule(self):
        self.datamodule = build_datamodule(
            self.worker, total_trajs=self.hparams.total_trajs
        )
        self.datamodule.collect_trajectories(prog=False)

    def make_lightning_trainer(self):
        logger = pl.loggers.WandbLogger(
            save_dir=self.run.dir, log_model=False, experiment=self.run
        )

        early_stopping = pl.callbacks.EarlyStopping(
            monitor=LightningModel.early_stop_on,
            min_delta=float(self.hparams.improvement_delta),
            patience=int(self.hparams.patience),
            mode="min",
            strict=True,
        )
        checkpointing = pl.callbacks.ModelCheckpoint(
            dirpath=osp.join(self.run.dir, "checkpoints"),
            monitor=LightningModel.early_stop_on,
            save_top_k=-1,
            period=10,
            save_last=True,
        )
        self.trainer = pl.Trainer(
            default_root_dir=self.run.dir,
            logger=logger,
            num_sanity_val_steps=2,
            callbacks=[early_stopping, checkpointing, InputStatistics()],
            max_epochs=self.hparams.max_epochs,
            progress_bar_refresh_rate=0,  # don't show progress bar for model training
            weights_summary=None,  # don't print summary before training
        )

    def make_artifact(self):
        env = self.worker.env
        self.artifact = wandb.Artifact(
            f"svg_prediction-lqg{env.n_state}.{env.n_ctrl}.{env.horizon}", type="model"
        )

    def step(self) -> dict:
        self.log_env_info()
        with utils.suppress_dataloader_warning():
            self.trainer.fit(self.model, datamodule=self.datamodule)

            results = self.trainer.test(self.model, datamodule=self.datamodule)[0]
            self.run.summary.update(results)

        self.artifact.add_dir(self.trainer.checkpoint_callback.dirpath)
        self.run.log_artifact(self.artifact)
        return {tune.result.DONE: True, **results}

    def log_env_info(self):
        dynamics = self.worker.env.dynamics
        eigvals = lqg_util.stationary_eigvals(dynamics)
        tests = {
            "stability": lqg_util.isstable(eigvals=eigvals),
            "controllability": lqg_util.iscontrollable(dynamics),
        }
        self.run.summary.update(tests)
        self.run.summary.update({"Fs_eigvals": wandb.Histogram(eigvals)})

    def cleanup(self):
        self.run.finish()

In [5]:
ray.init(logging_level=logging.WARNING)
lqsvg.register_all()
utils.suppress_lightning_info_logging()

config = {
    "env_config": dict(
        n_state=2,
        n_ctrl=2,
        horizon=100,
        stationary=True,
        Fs_eigval_range=(0.0, 1.0),
        num_envs=100,
    ),
    "policy": {"initialization": "xavier_uniform"},
    "learning_rate": 1e-3,
    "mc_samples": 32,
    "total_trajs": 1000,
    "improvement_delta": 0.0,
    "patience": 3,
    "max_epochs": 200,
}

analysis = tune.run(Experiment, config=config, num_samples=4, local_dir="./results")

Trial name,status,loc
Experiment_6dbf9_00000,RUNNING,


(pid=13612) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=13615) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=13613) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=13614) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=13612) wandb: Tracking run with wandb version 0.10.23
(pid=13612) wandb: Syncing run SVG Prediction
(pid=13612) wandb: ⭐️ View project at https://wandb.ai/angelovtt/LQG-SVG
(pid=13612) wandb: 🚀 View run at https://wandb.ai/angelovtt/LQG-SVG/runs/1nrdaeun
(pid=13612) wandb: Run data is saved locally in /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/wandb/run-20210324_140743-1nrdaeun
(pid=13612) wandb: Run `wandb offline` to turn off syncing.
(pid=13614) wandb: Tracking run with wandb version 0.10.23
(pid=13614) wandb: Syncing run SVG Prediction
(pid=13614) wandb: ⭐️ V

(pid=13612) 
(pid=13614) 
(pid=13615) 
(pid=13613) 


(pid=13612) 2021-03-24 14:07:46,955	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=13615) 2021-03-24 14:07:46,995	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=13613) 2021-03-24 14:07:46,978	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=13614) 2021-03-24 14:07:46,960	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=13612) GPU available: False, used: False
(pid=13612) TPU available: None, using: 0 TPU cores
(pid=13612) 2021-03-24 14:08:10,027	INFO trainable.py:100 -- Trainable.setup took 27.533 seconds. If your trainable is slow to initialize, consider se

(pid=13612) --------------------------------------------------------------------------------
(pid=13612) DATALOADER:0 TEST RESULTS
(pid=13612) {'test/analytic_cossim': tensor(0.1866),
(pid=13612)  'test/analytic_diff': tensor(17.0537),
(pid=13612)  'test/analytic_svg_norm': tensor(9697.0586),
(pid=13612)  'test/analytic_value': tensor(-5062.4160),
(pid=13612)  'test/loss': tensor(351.4277),
(pid=13612)  'test/monte_carlo_cossim': tensor(0.1868),
(pid=13612)  'test/monte_carlo_diff': tensor(-132.6631),
(pid=13612)  'test/monte_carlo_svg_norm': tensor(10097.2988),
(pid=13612)  'test/monte_carlo_value': tensor(-5212.1328),
(pid=13612)  'true_svg_norm': tensor(1673.1217),
(pid=13612)  'true_value': tensor(-5079.4697)}
(pid=13612) --------------------------------------------------------------------------------
Result for Experiment_6dbf9_00003:
  date: 2021-03-24_14-12-11
  done: true
  experiment_id: f207bbef87314670b3882e1a03db8637
  hostname: Angelos-MBP
  iterations_since_restore: 1
  n

...

wandb:                                                                                
(pid=13614) wandb: Find user logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/wandb/run-20210324_140743-2uceg1h7/logs/debug.log
(pid=13614) wandb: Find internal logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/wandb/run-20210324_140743-2uceg1h7/logs/debug-internal.log
(pid=13614) wandb: Run summary:
(pid=13614) wandb:                   stability True
(pid=13614) wandb:             controllability True
(pid=13614) wandb:                    val/loss 442.57678
(pid=13614) wandb:       val/monte_carlo_value -3661.51758
(pid=13614) wandb:    val/monte_carlo_svg_norm 5190.45068
(pid=13614) wandb:          val/analytic_value -3404.2644
(pid=13614) wandb:       val/analytic_svg_norm 4690.47559
(pid=13614) wandb:        val/monte_carlo_diff 7905.76953
(pid=13614) wandb:           val/analytic_diff 8163.02246
(pid=13614) wandb:

(pid=13614) 


(pid=13615) wandb: Find user logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/wandb/run-20210324_140743-26rc648x/logs/debug.log
(pid=13615) wandb: Find internal logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/wandb/run-20210324_140743-26rc648x/logs/debug-internal.log
(pid=13615) wandb: Run summary:
(pid=13615) wandb:                   stability True
(pid=13615) wandb:             controllability True
(pid=13615) wandb:                    val/loss 236.32231
(pid=13615) wandb:       val/monte_carlo_value -691.99103
(pid=13615) wandb:    val/monte_carlo_svg_norm 685.31439
(pid=13615) wandb:          val/analytic_value -681.23676
(pid=13615) wandb:       val/analytic_svg_norm 668.13129
(pid=13615) wandb:        val/monte_carlo_diff -19.28796
(pid=13615) wandb:           val/analytic_diff -8.53369
(pid=13615) wandb:      val/monte_carlo_cossim 0.22875
(pid=13615) wandb:         val/analytic_cossim 0.22681
(

(pid=13615) 


wandb:                                                                                
(pid=13613) wandb: Find user logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/wandb/run-20210324_140743-3pepdaao/logs/debug.log
(pid=13613) wandb: Find internal logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/wandb/run-20210324_140743-3pepdaao/logs/debug-internal.log
(pid=13613) wandb: Run summary:
(pid=13613) wandb:                   stability True
(pid=13613) wandb:             controllability True
(pid=13613) wandb:                    val/loss 303.50375
(pid=13613) wandb:       val/monte_carlo_value -599.3916
(pid=13613) wandb:    val/monte_carlo_svg_norm 186.22202
(pid=13613) wandb:          val/analytic_value -647.32672
(pid=13613) wandb:       val/analytic_svg_norm 222.30438
(pid=13613) wandb:        val/monte_carlo_diff 51.15283
(pid=13613) wandb:           val/analytic_diff 3.21771
(pid=13613) wandb:      val

In [6]:
ray.shutdown()